In [1]:
!pip install numpy scipy

In [36]:
import numpy as np
from scipy.io import wavfile
from scipy import signal
from IPython.display import Audio

def realistic_car_passby(velocity=90.0, base_freq=100.0, duration=12.0, sr=44100):
    """
    More realistic car pass-by with:
    - Engine harmonics
    - Road/tire noise
    - Wind hiss
    - Doppler shift
    - Distance attenuation
    - Stereo panning + reverb
    """
    c = 343.0  # speed of sound
    t = np.linspace(0, duration, int(sr*duration), endpoint=False)
    dt = 1/sr
    pass_time = duration/2.0  # midpoint = closest approach

    # ---------------------------
    # Doppler-shifted engine frequency
    # ---------------------------
    v_rel = np.where(t <= pass_time, velocity, -velocity)
    rpm_curve = base_freq * (1 + 0.4 * (t/duration))  # RPM ramp
    base_inst_freq = rpm_curve * (c / (c - v_rel))    # Doppler effect

    jitter = 1.0 + 0.004*np.random.randn(len(t))  # subtle vibration
    inst_freq = base_inst_freq * jitter
    inst_phase = np.cumsum(2*np.pi*inst_freq*dt)

    # Engine harmonics
    orders = [1, 2, 3, 4, 5, 6]
    amps   = [1.0, 0.6, 0.4, 0.25, 0.15, 0.1]
    harmonics = sum(a*np.sin(k*inst_phase) for k, a in zip(orders, amps))

    # Low engine rumble
    rumble = 0.35*np.sin(2*np.pi*18*t + 0.5*np.sin(2*np.pi*2*t))

    # Road noise (low/mid broadband, like rolling tires)
    road_noise = np.random.randn(len(t)) * 0.015 * (np.abs(v_rel)/velocity)
    b, a = signal.butter(4, 300/(sr/2), btype="low")
    road_noise = signal.lfilter(b, a, road_noise)

    # Wind noise (broadband hiss, high-passed)
    wind = np.random.randn(len(t)) * 0.01 * (np.abs(v_rel)/velocity)
    b, a = signal.butter(4, 500/(sr/2), btype="highpass")
    wind = signal.lfilter(b, a, wind)

    # Combine components
    engine = harmonics + rumble + road_noise + wind

    # Resonance shaping (exhaust pipe effect)
    for f0 in [180, 600, 1200, 2500]:
        b, a = signal.iirpeak(f0/(sr/2), Q=10)
        engine = signal.lfilter(b, a, engine)

    # ---------------------------
    # Distance attenuation
    # ---------------------------
    forward_dist = np.where(t <= pass_time,
                            velocity*(pass_time - t),
                            velocity*(t - pass_time))
    lateral_offset = 3.0  # meters
    dist = np.sqrt(forward_dist**2 + lateral_offset**2)

    att = 1.0 / (0.2 + dist/4.0)
    engine *= att

    # ---------------------------
    # Stereo panning
    # ---------------------------
    pan = np.tanh((t - pass_time) / (duration/10))
    left = engine * np.sqrt(0.5*(1 - pan))
    right = engine * np.sqrt(0.5*(1 + pan))

    # Small random imbalance between channels
    left *= (1 + 0.02*np.random.randn(len(t)))
    right *= (1 + 0.02*np.random.randn(len(t)))

    stereo = np.vstack([left, right]).T.astype(np.float32)

    # ---------------------------
    # Light reverb (outdoor echo)
    # ---------------------------
    ir = np.zeros(int(sr*0.25))
    ir[0] = 1
    ir[int(sr*0.05)] = 0.25
    ir[int(sr*0.1)] = 0.15
    ir[int(sr*0.18)] = 0.08
    for ch in range(2):
        reverb = signal.fftconvolve(stereo[:,ch], ir)[:len(stereo)]
        stereo[:,ch] = 0.75*stereo[:,ch] + 0.25*reverb

    # Normalize
    stereo /= np.max(np.abs(stereo)) + 1e-6
    return sr, stereo

# ▶ Example
sr, snd = realistic_car_passby(velocity=60.0, base_freq=110.0, duration=12.0)
wavfile.write("realistic_car_passby.wav", sr, (snd*32767).astype(np.int16))
Audio("realistic_car_passby.wav")
